In [1]:
from google.cloud import bigquery
import os
import pandas as pd

GCP_PROJECT = 'opensource-observer'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '../../../oso_gcp_credentials.json'
client = bigquery.Client(GCP_PROJECT)

In [2]:
query = """

select
  onchain_builders.project_name as `Project Name`,
  dev_tools.project_name as `Dependency`,
  sum(onchain_metrics.transaction_count_6_months) as `Project Total Txns`,
  sum(onchain_metrics.active_contract_count_90_days) / 3 as `Project Avg MAUs`,
  sum(onchain_metrics.gas_fees_sum_6_months) as `Project Total Gas Fees`,
from `oso_production.sboms_v0` as sboms
join `oso_production.projects_by_collection_v1` as onchain_builders
  on sboms.from_project_id = onchain_builders.project_id
join `oso_production.package_owners_v0` as pkgs
  on
    sboms.to_package_artifact_name = pkgs.package_artifact_name
    and sboms.to_package_artifact_source = pkgs.package_artifact_source
join `oso_production.projects_v1` as dev_tools
  on pkgs.package_owner_project_id = dev_tools.project_id
join `oso_production.onchain_metrics_by_project_v1` as onchain_metrics
  on onchain_builders.project_id = onchain_metrics.project_id
where
  onchain_builders.collection_name = 'op-retrofunding-4'
  and onchain_metrics.event_source in ('OPTIMISM', 'BASE', 'MODE', 'ZORA')
  and dev_tools.project_id in (
    select project_id from `oso_production.projects_by_collection_v1`
    where collection_name = 'op-rpgf3'
  )
group by 1,2
"""

result = client.query(query)
df = result.to_dataframe().sort_values(by='Project Total Txns', ascending=False)
df = df[df['Project Total Txns'] > 100]
df.reset_index(inplace=True, drop=True)
df.head()

,Project Name,Dependency,Project Total Txns,Project Avg MAUs,Project Total Gas Fees
0,zora,ethers-io,8.572884e+09,2.648632e+07,17305.571616
1,layer-zero,ethers-io,3.860114e+09,1.201600e+04,5996.633767
2,across,ethers-io,1.200631e+09,6.013333e+03,11163.584370
3,lifinance,ethers-io,1.135957e+09,1.390667e+03,3335.429666
4,sushi,ethers-io,1.018108e+09,7.716000e+04,5076.661306


In [3]:
df.to_csv('data/prelim_weighted_graph.csv')